In [97]:
from openai import OpenAI
import json
from dotenv import load_dotenv,find_dotenv

_ : bool = load_dotenv(find_dotenv())

In [98]:
client  : OpenAI = OpenAI()

### Step 0 : Define functions
First, Define your functions:

In [99]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

### Step 1 : Create an Assistant and register / report your functions



In [100]:
import json
def show_json(message,obj):
    display(message,json.loads(obj.model_dump_json()))

In [101]:
from openai.types.beta import Assistant
assistant : Assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided function to answer questions.",
    model= "gpt-3.5-turbo-1106",
    tools=[{
        "type":"function",
        "function" : {
            "name" : "getCurrentWeather",
            "description" : "Get the weather in location",
            "parameters" : {
                "type" : "object",
                "properties": {
                    "location" : {"type" : "string", "description": "The city and state e.g San Francisco,SA"},
                    "unit": {"type" : "string","enum" : ["c","f"]}
                },
                "required" : ["location"]
            }
        }
    },
      {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco"},
        },
        "required": ["location"]
      }
    } }
    ]
)

### Step 2 : Create a Thread

In [102]:
from openai.types.beta.thread import Thread

thread : Thread =  client.beta.threads.create()
show_json("thread", thread)

'thread'

{'id': 'thread_DMGa6jm0tLWplRNRW2mT3yjG',
 'created_at': 1705125412,
 'metadata': {},
 'object': 'thread'}

### Step 3 : Add a Message to a Thread


In [103]:
from openai.types.beta.threads.thread_message import ThreadMessage
# First Request
message : ThreadMessage = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= "How is the weather in Paris?"
)




In [104]:
show_json("Message", message)

'Message'

{'id': 'msg_J09ThSreMdHxKFqmXannDWsH',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'How is the weather in Paris?'},
   'type': 'text'}],
 'created_at': 1705125413,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_DMGa6jm0tLWplRNRW2mT3yjG'}

### Step 4 : Run the Assistant

In [105]:
from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [106]:
show_json("Run",run)

'Run'

{'id': 'run_Zt6N6jMIEscCGfhHDt7ATtil',
 'assistant_id': 'asst_IsS7pXdtTfCcRBmioTveVum4',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1705125413,
 'expires_at': 1705126013,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided function to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_DMGa6jm0tLWplRNRW2mT3yjG',
 'tools': [{'function': {'name': 'getCurrentWeather',
    'description': 'Get the weather in location',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string',
       'description': 'The city and state e.g San Francisco,SA'},
      'unit': {'type': 'string', 'enum': ['c', 'f']}},
     'required': ['location']}},
   'type': 'function'},
  {'function': {'name': 'getNickname',
    'description': 'Get the nickname of a city',
    'parameter

In [107]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname" : getNickname
}

### Step 5 : Polling for Updates and Calling Functions


In [109]:
import time

while True:
    runstatus = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    if runstatus.status =="requires_action":
        toolCalls = runstatus.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        for toolcall in toolCalls:
            function_name = toolcall.function.name
            function_args = json.loads(toolcall.function.arguments)
            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                if function_name =="getCurrentWeather":
                    response = function_to_call(location=function_args.get("location"),unit=function_args.get("unit"))
                    tool_outputs.append({
                        "tool_call_id" : toolcall.id,
                        "output": response
                    })
                elif function_name =="getNickname":
                    response = function_to_call(location=function_args.get("location"))
                    tool_outputs.append({
                        "tool_call_id" : toolcall.id,
                        "output": response
                    })


completed...........logic
Assistant: The current weather in Paris is 22°C and it's known as "The City of Light".

User: How is the weather in Paris?

